# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [1]:
import logging

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [4]:
tracks = pd.read_parquet("tracks.parquet")
catalog_names = pd.read_parquet("catalog_names.parquet")
interactions = pd.read_parquet("interactions.parquet")

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [12]:
print("--------- Tracks Info:")
tracks.info()
print("\n--------- Catalog_names Info:")
catalog_names.info()
print("\n--------- Interactions Info:")
interactions.info()

--------- Tracks Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB

--------- Catalog_names Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB

--------- Interactions Info:
<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   i

In [20]:
print(interactions[["user_id", "track_id", "started_at", "track_seq"]].sample(5, random_state=52).sort_values("user_id").set_index(["user_id", "track_seq"]).to_string())

                   track_id started_at
user_id track_seq                     
46179   37         29544272 2022-11-23
70117   1055       28213677 2022-09-18
72819   270        85090862 2022-08-08
471152  18            55682 2022-09-10
688081  211          220497 2022-02-22


In [15]:
tracks.sample(3).T

,121259,124414,650583
track_id,2190993,2214833,45838742
albums,"[216815, 8828290, 13888175, 17134314]",[63387],"[6161725, 8854149]"
artists,[4029],[41110],[4435372]
genres,"[26, 74]","[11, 20]",[11]


In [16]:
catalog_names.sample(3).T

,511549,299,361165
id,11574627,666,6207395
type,album,album,album
name,Gerek,Sabotage,На самокате


In [17]:
interactions.sample(3).T

,788,76,1
user_id,871257,1133456,59724
track_id,50868869,79791789,1535858
track_seq,789,77,2
started_at,2022-10-17 00:00:00,2022-10-22 00:00:00,2022-05-25 00:00:00


In [19]:
print("Пропуски в tracks:")
print(tracks.isna().sum(), "\n")

print("Пропуски в interactions:")
print(interactions.isna().sum(), "\n")

print("Пропуски в catalog_names:")
print(catalog_names.isna().sum(), "\n")

Пропуски в tracks:
track_id    0
albums      0
artists     0
genres      0
dtype: int64 

Пропуски в interactions:
user_id       0
track_id      0
track_seq     0
started_at    0
dtype: int64 

Пропуски в catalog_names:
id      0
type    0
name    0
dtype: int64 



In [ ]:
# Проверка пересечения данных

missing_tracks = set(interactions.track_id.unique()) - set(tracks.track_id.unique())
len(missing_tracks)

0

In [23]:
# количество уникальных пользователей и треков
n_users = interactions.user_id.nunique()
n_tracks = tracks.track_id.nunique()

print(n_users, n_tracks)

1373221 1000000


Первоначальные исправления:

In [22]:
# Приводим даты к типу date
interactions["started_at"] = interactions["started_at"].dt.date

# Проверяем согласованность track_id
interactions = interactions[interactions.track_id.isin(tracks.track_id)]

# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

1. **Типы идентификаторов:**

- user_id и track_id представлены в числовых типах (int32/int64).
- Приведение типов не требуется.

2. **Проверка качества данных:**

- *Tracks:*
    - Нет пропусков.
    - Неизвестных альбомов, артистов и жанров нет, т.к. все значения заполнены.

- *Catalog_names:*
    - Нет пропусков. 
    - Никаких проблем не обнаружено.

- *Interactions:*
    - Более 222 млн событий. 
    - Нет пропусков. 
    - Типы колонок корректны.
    - Дата приведена к типу date (исправление применено).

3. **Проверка пересечения данных:**

- Все события в interactions корректно ссылаются на существующие треки.

4. **Объёмы данных:**

- Пользователей: 1 373 221
- Треков: 1 000 000

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.